In [ ]:
#pip install affine, s3fs, tifffile, zarr, python-dotenv

In [ ]:
import os, time
from itertools import chain
from affine import Affine
from botocore import UNSIGNED
from dotenv import dotenv_values, load_dotenv
import numpy as np
import pandas as pd
from tifffile import imread
from tifffile.tifffile import TiffFile
from typing import List, Tuple
import zarr, s3fs


def get_geotiff_meta_data(path, s3):
    with s3.open(path) as f:
        with TiffFile(f) as tif:
            scale: Tuple[float, float, float] = tif.geotiff_metadata["ModelPixelScale"]
            tie_point: List[float] = tif.geotiff_metadata["ModelTiepoint"]
            store = tif.series[0].aszarr()
            zarray = zarr.open(store, mode='r')
            shape: List[int] = tif.series[0].shape
            i, j, k, x, y, z = tie_point[0:6]
            sx, sy, sz = scale
            transform = Affine(sx, 0.0, x - i * sx, 0.0, -sy, y + j * sy)
            return shape, transform

def get_coordinates(longitudes, latitudes, transform):
    coords = np.vstack((longitudes, latitudes, np.ones(len(longitudes))))
    inv_trans = ~transform
    mat = np.array(inv_trans).reshape(3,3)
    frac_image_coords = mat @ coords
    image_coords = np.floor(frac_image_coords).astype(int)
    return image_coords

def zarr_create(group_path, array_path, s3, shape, transform, return_periods):
    """
    Create zarr with given shape and affine transform
    """
    store = s3fs.S3Map(root = group_path, s3 = s3, check = False)
    root = zarr.open(store=store, mode = 'w') # open group as root
    z = root.create_dataset(array_path, shape=(len(return_periods), shape[0], shape[1]), chunks=(len(return_periods), 1000, 1000), dtype='f4') # array_path interpreted as path within group
    trans_members = [transform.a, transform.b, transform.c, transform.d, transform.e, transform.f]
    mat3x3 = [x * 1.0 for x in trans_members] + [0.0, 0.0, 1.0]
    z.attrs['transform_mat3x3'] = mat3x3
    z.attrs['index_values'] = return_periods
    z.attrs['index_name'] = "return period (years)"
    return z

def zarr_remove(group_path, array_path, s3):
    """
    Create zarr with given shape and affine transform
    """
    store = s3fs.S3Map(root = group_path, s3 = s3, check = False)
    root = zarr.open(store=store, mode = 'w') # open group as root
    z = root.pop(array_path)
    
def zarr_write(src_path, src_s3, dest_zarr, index):
    """
    Writes data from GeoTiff sepecified by src_path and src_s3 S3FileSystem to destination zarr array dest_zarr
    putting data into index.
    """
    with src_s3.open(src_path) as f:
        with TiffFile(f) as tif:
            store = tif.series[0].aszarr()
            z_in = zarr.open(store, mode='r')
            dest_zarr[index, :, :] = z_in[:, :]


In [ ]:
from dotenv import dotenv_values, load_dotenv
import os
import pathlib

dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

In [ ]:
# source
#"000000000WATCH"
circ_models = ["00000NorESM1-M", "0000GFDL-ESM2M", "0000HadGEM2-ES", "00IPSL-CM5A-LR", "MIROC-ESM-CHEM"]
years = ["2030", "2050", "2080"]
rcps = ["rcp4p5", "rcp8p5"]
#circ_models = ["0000GFDL-ESM2M", "0000HadGEM2-ES", "00IPSL-CM5A-LR", "MIROC-ESM-CHEM"]

src_bucket = "wri-projects"
src_prefix = "AqueductFloodTool/download/v2"
src_returns = [5, 10, 25, 50, 100, 250, 500, 1000]

# destination
dest_bucket = "redhat-osc-physical-landing-647521352890"
dest_prefix = "hazard"

# no authentication for source currently
s3_source = s3fs.S3FileSystem(
                config_kwargs=dict(signature_version=UNSIGNED))

# need landing credentials for target
s3_dest = s3fs.S3FileSystem(anon=False,
                key=os.environ['S3_LANDING_ACCESS_KEY'],
                secret=os.environ['S3_LANDING_SECRET_KEY'])

for rcp in rcps:
    print("RCP: " + rcp)
    for year in years:
        print("Year: " + year)
        for circ_model in circ_models:
            src_filenames = ["inun{0}_{1}_{2}_{3}_rp{4:05d}".format("river", rcp, circ_model, year, i) for i in src_returns] 
            dest_filename = "inun{0}_{1}_{2}_{3}".format("river", rcp, circ_model, year) 

            # get meta-data from first in list
            shape, transform = get_geotiff_meta_data(os.path.join(src_bucket, src_prefix, src_filenames[0] + ".tif"), s3_source)

            z = zarr_create(os.path.join(dest_bucket, dest_prefix, "hazard.zarr"),
                os.path.join("inundation/wri/v2/", dest_filename),
                s3_dest,
                shape,
                transform,
                list([r * 1.0 for r in src_returns]))

            print('Destination: ' + dest_filename)
            for (i, filename) in enumerate(src_filenames):
                print(f'File {i + 1}/{len(src_filenames)}', end = '...')
                zarr_write(os.path.join(src_bucket, src_prefix, filename + ".tif"), s3_source, z, i)


In [ ]:
# some useful utilities

zarr_remove(os.path.join(dest_bucket, dest_prefix, "hazard.zarr"),
    os.path.join("inundation/wri/v2/", dest_filename),
    s3_dest)
s3_dest.ls('redhat-osc-physical-landing-647521352890/hazard/hazard.zarr/inundation/wri/v2')
with s3_dest.open('redhat-osc-physical-landing-647521352890/hazard/hazard.zarr/inundation/wri/v2/inunriver_rcp8p5_MIROC-ESM-CHEM_2080/.zarray') as r:
    a = r.read()
print(a)

In [ ]:
# test
import rasterio
import json
with open('coords.json', 'r') as f:
    coords = json.loads(f.read())
# test new zarr versus rasterio

longitudes = np.array(coords['longitudes'])[0:1000]
latitudes = np.array(coords['latitudes'])[0:1000]

res = []
for (i, filename) in enumerate(src_filenames):
    with s3_source.open(os.path.join(src_bucket, src_prefix, filename + ".tif")) as f:
        with rasterio.open(f) as dataset:
            points = [[lon, lat] for (lon, lat) in zip(longitudes, latitudes)]
            samples_rasterio = np.array(list(rasterio.sample.sample_gen(dataset, points)))
            samples_rasterio.resize([len(longitudes)])
            res.append(samples_rasterio)


In [ ]:
store = s3fs.S3Map(root = os.path.join(dest_bucket, dest_prefix, "hazard.zarr"), s3 = s3_dest, check = False)
root = zarr.open(store, mode='r')
z = root[os.path.join('inundation/wri/v2/', dest_filename)]

In [ ]:
import time
image_coords = get_coordinates(longitudes, latitudes, transform)
cz = np.tile(np.arange(8), image_coords.shape[1]) 
cx = np.repeat(image_coords[1,:], 8)
cy = np.repeat(image_coords[0,:], 8)

start = time.time()
data = z.get_coordinate_selection((cz, cx, cy))
stop = time.time()
d = data.reshape([len(longitudes), 8])
print(stop-start)

In [ ]:
longitudes = np.array(coords['longitudes'])[0:10000]
latitudes = np.array(coords['latitudes'])[0:10000]

In [ ]:
for i in range(8):
    print(np.max(np.abs(d[:, i] - res[i])))